In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import sys
import os
import glob
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.magnum.magopter as mg
import flopter.core.constants as c
import flopter.magnum.database as mdb
import flopter.core.normalisation as nrm
import flopter.utils.lputils as lp
import flopter.classes.ivdata as ivd
import flopter.core.filtering as filt
import scipy.signal as sig
import flopter.core.fitters as fts

ModuleNotFoundError: No module named 'flopter.core.magopter'

**FILE IO**

In [1]:
folders = ['2018-05-01_Leland/', '2018-05-02_Leland/', '2018-05-03_Leland/',
           '2018-06-05_Leland/', '2018-06-06_Leland/', '2018-06-07_Leland/']
files = []
file_folders = []
for folder1 in folders:
    os.chdir(mg.Magopter.get_data_path() + folder1)
    files.extend(glob.glob('*.adc'))
    file_folders.extend([folder1] * len(glob.glob('*.adc')))
files.sort()

# file = files[286]
adc_file = files[285]
ts_file = files[284]
folder = file_folders[-2]
print('"{}" \t\t "{}" \t\t "{}"'.format(folder, adc_file, ts_file))

mp = lp.MagnumProbes()
probe = mp.probe_s

NameError: name 'os' is not defined

In [3]:
for adc_f in files[285:297]:
    print(adc_file)
    
files_of_interest = files[285:297]

2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc
2018-06-07 14h 38m 33s TT_06564321037878221702.adc


In [4]:
dsr = 1

# Create magopter object
magopter = mg.Magopter(folder, adc_file, ts_filename=ts_file)
magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, plot_fl=False, crit_freq=None, crit_ampl=None)

print('0: {}, 1: {}'.format(len(magopter.iv_arrs[0]), len(magopter.iv_arrs[1])))

# Get values for TS measurements, depending on whether Magnum database is available.
magnum_access_fl = magopter.ts_temp is not None
if magnum_access_fl:
    temps = [np.max(temp) / nrm.ELEM_CHARGE for temp in magopter.ts_temp[mdb.DATA]]
    denss = [np.max(dens) for dens in magopter.ts_dens[mdb.DATA]]
    T_e_ts = np.mean(temps)
    d_T_e_ts = np.std(temps) / np.sqrt(len(temps))
    n_e_ts = np.mean(denss)
    d_n_e_ts = np.std(denss) / np.sqrt(len(denss))
else:
    T_e_ts = 1.61
    d_T_e_ts = 0.01
    n_e_ts = 1.4e20
    d_n_e_ts = 0.1e20

Ice.ConnectionRefusedException:
Connection refused
Database could not be connected to, operating in offline mode.


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


peaks_len = 798
0: 798, 1: 798


**Xarray dataset creation + plotting**

In [5]:
# print length of the 0th probes (probe S) number of sweeps
print(len(magopter.iv_arrs[0]))

# Create relative t array by subtracting the first timestep value from the first time array 
first_time_arr = magopter.iv_arrs[0][0]['t']
relative_t = np.zeros(len(first_time_arr))
print(np.shape(relative_t))

relative_t = first_time_arr - first_time_arr[0]

798
(5000,)


In [8]:
# create a list of datasets for each sweep
ds_list = []
for i, iv in enumerate(magopter.iv_arrs[0]):
    if i % 2 == 0:
        ds = xr.Dataset({'voltage': (['time'], iv['V'][:2500]),
                         'current': (['time'], iv['I'][:2500]),
                         'shot_time': (['time'], iv['t'][:2500]),
                         'start_time': iv['t'][0]},
                        coords={'time': relative_t[:2500], 'direction': 'up'})
    else:
        ds = xr.Dataset({'voltage': (['time'], np.flip(iv['V'][:2500])),
                         'current': (['time'], np.flip(iv['I'][:2500])),
                         'shot_time': (['time'], np.flip(iv['t'][:2500])),
                         'start_time': iv['t'][0]},
                        coords={'time': relative_t[:2500], 'direction': 'down'})
    ds_list.append(ds)

# # Separate into up and down sweeps then concat along sweep direction as an axis
print(len(ds_list[::2]), len(ds_list[1::2]))
ds_ups = xr.concat(ds_list[::2], 'sweep')
ds_downs = xr.concat(ds_list[1::2], 'sweep')

direction = xr.DataArray(np.array(['up', 'down']), dims=['direction'], name='direction')
ds_full = xr.concat([ds_ups, ds_downs], dim=direction)

if not magnum_access_fl:
    ds_full.max(dim='time').mean('direction')['current'].plot.line(x='sweep')
    ds_full.max(dim='time').mean('direction').isel(sweep=slice(100,300))['current'].plot.line(x='sweep')

    # Choose only the IVs in the static section
    ds_full = ds_full.isel(sweep=slice(100,300))

# Average across each sweep direction
sweep_avg_up = ds_full.sel(direction = 'up').mean('sweep')
sweep_avg_dn = ds_full.sel(direction = 'down').mean('sweep')

# Add in standard deviation of each bin as a new data variable 
sweep_avg_up = sweep_avg_up.assign({'d_current': ds_full.sel(direction = 'up').std('sweep')['current']})
sweep_avg_dn = sweep_avg_dn.assign({'d_current': ds_full.sel(direction = 'down').std('sweep')['current']})

print(ds_full)

399 399
<xarray.Dataset>
Dimensions:     (direction: 2, sweep: 200, time: 2500)
Coordinates:
  * time        (time) float64 0.0 1e-05 2e-05 3e-05 ... 0.02497 0.02498 0.02499
  * direction   (direction) <U4 'up' 'down'
Dimensions without coordinates: sweep
Data variables:
    voltage     (direction, sweep, time) float64 -97.07 -97.26 ... 5.345 5.535
    current     (direction, sweep, time) float64 1.06 1.077 ... -1.882 -1.899
    shot_time   (direction, sweep, time) float64 6.022 6.022 6.022 ... 16.0 16.0
    start_time  (direction, sweep) float64 6.022 6.072 6.122 ... 15.9 15.95 16.0


In [7]:
ds_full.max(dim='time').mean('direction')['current'].plot.line(x='sweep')
ds_full.max(dim='time').mean('direction').isel(sweep=slice(100,300))['current'].plot.line(x='sweep')

# ds_full = ds_full.isel(sweep=slice(100,300))

In [141]:
print(ds_full)
capacitance_ds = (ds_full.sel(direction = 'up') - ds_full.sel(direction = 'down')).mean('sweep')
# capacitance_ds = ds_full.diff('direction').mean('sweep').sel(direction = 'down')
# capacitance_ds = capacitance_ds.assign({'d_current': ds_full.diff('direction').std('sweep')['current']})
print(f'\n{capacitance_ds}')

plt.figure()
plt.plot(capacitance_ds['time'], capacitance_ds['current'])

<xarray.Dataset>
Dimensions:     (direction: 2, sweep: 200, time: 2500)
Coordinates:
  * time        (time) float64 0.0 1e-05 2e-05 3e-05 ... 0.02497 0.02498 0.02499
  * direction   (direction) <U4 'up' 'down'
Dimensions without coordinates: sweep
Data variables:
    voltage     (direction, sweep, time) float64 -97.07 -97.26 ... 5.345 5.535
    current     (direction, sweep, time) float64 1.06 1.077 ... -1.882 -1.899
    shot_time   (direction, sweep, time) float64 6.022 6.022 6.022 ... 16.0 16.0
    start_time  (direction, sweep) float64 6.022 6.072 6.122 ... 15.9 15.95 16.0

<xarray.Dataset>
Dimensions:     (time: 2500)
Coordinates:
  * time        (time) float64 0.0 1e-05 2e-05 3e-05 ... 0.02497 0.02498 0.02499
Data variables:
    voltage     (time) float64 0.02402 0.1869 0.2986 ... 0.2029 0.1558 0.001687
    current     (time) float64 0.0006489 0.002725 ... -0.003417 -0.001254
    shot_time   (time) float64 -0.04999 -0.04997 -0.04995 ... -3e-05 -1e-05
    start_time  float64 -0.025

In [43]:
plt.figure()
plt.errorbar(sweep_avg_up['time'], sweep_avg_up['current'], yerr=sweep_avg_up['d_current'], 
             ecolor='silver')

plt.figure()
plt.errorbar(sweep_avg_dn['time'], sweep_avg_dn['current'], yerr=sweep_avg_dn['d_current'], 
             ecolor='silver')
plt.show()

**Capacitance investigation**

In [40]:
# Make plot of up-sweep averaged current minus down-sweep averaged current 
plt.subplot(221)
plt.plot(sweep_avg_up['time'], sweep_avg_up['current'])
plt.plot(sweep_avg_dn['time'], sweep_avg_dn['current'])

plt.subplot(223)
plt.plot(sweep_avg_up['time'].data, sweep_avg_up['current'].data - sweep_avg_dn['current'].data, color='red')

plt.subplot(222)
plt.plot(sweep_avg_up['time'], sweep_avg_up['voltage'])
plt.plot(sweep_avg_dn['time'], sweep_avg_dn['voltage'])

plt.subplot(224)
plt.plot(sweep_avg_up['time'].data, sweep_avg_up['voltage'].data - sweep_avg_dn['voltage'].data, color='red')
plt.show()

In [41]:
# Look at the difference between the up-sweep and down-sweep voltage 
capacitance = sweep_avg_up['voltage'].data - sweep_avg_dn['voltage'].data
capacitance_shifted = sweep_avg_up['voltage'].data[0:-1] - sweep_avg_dn['voltage'].data[1:]

fig, ax = plt.subplots()
plt.plot(sweep_avg_up['time'], sweep_avg_up['voltage'].data)
plt.plot(sweep_avg_dn['time'], np.flip(sweep_avg_dn['voltage'].data))

# ax1 = ax.twinx()
plt.figure()
plt.plot(sweep_avg_up['voltage'], capacitance, color='red')
plt.plot(sweep_avg_up['voltage'][0:-1], capacitance_shifted, color='blue')
plt.show()

In [42]:
# Run an fft on the current hysteresis
cap_current = sweep_avg_up['current'].data - np.flip(sweep_avg_dn['current'].data)
time_series = sweep_avg_up['time'].data

spectrum = np.fft.fft(cap_current)
freq = np.fft.fftfreq(len(cap_current), time_series[1] - time_series[0])
amplitudes = np.abs(spectrum) / np.max(np.abs(spectrum))

plt.subplot(211)
plt.plot(time_series, cap_current)
plt.ylabel(r'$I_{up} - I_{down}$ / A')
plt.xlabel('Time through shot (t) / s')

# np.abs(spectrum[:]) / np.max(np.abs(spectrum[:]))
plt.subplot(212)
plt.semilogy(freq[:], amplitudes, '+', label='FFT Spectrum')
# plt.axhline(y=crit_ampl, linewidth=1.0, linestyle='dotted', color='red')
plt.xlabel('Frequency / Hz')
plt.ylabel('Amplitude')
plt.grid(which='both', axis='both')
plt.xlim(0, 50000)
plt.ylim(np.min(amplitudes) * 0.8, 1.2)
plt.legend()
plt.show()

__FITTING TIME__

In [25]:
# concatenate the up and down sweeps together to cancel the (small) capacitance effect
iv_data = ivd.IVData(np.concatenate([sweep_avg_up['voltage'].data, np.flip(sweep_avg_dn['voltage'].data)]), 
                     np.concatenate([sweep_avg_up['current'].data, np.flip(sweep_avg_dn['current'].data)]),
                     relative_t, 
                     sigma=np.concatenate([sweep_avg_up['d_current'].data, np.flip(sweep_avg_dn['d_current'].data)]), 
                     estimate_error_fl=False)

fit_data = iv_data.multi_fit(plot_fl=True)
# fit_data_trim = iv_data_trim.multi_fit(plot_fl=True)

In [10]:
# Try multifitting with a 3-parameter fit
simple_iv_fitter = fts.SimpleIVFitter()
iv_data.multi_fit(plot_fl=True, fitter=simple_iv_fitter)

In [26]:
plt.figure()
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data['sigma'], 
             ecolor='silver')
plt.show()

In [156]:
plt.figure()
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='r', zorder=10)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], ecolor='silver', label='Raw')
plt.legend()
plt.show()

In [181]:
#Try a single trim
iv_data_trim = ivd.IVData.non_contiguous_trim(iv_data, np.where(iv_data['I'] >= 0.8))
fit_data_trim = iv_data_trim.multi_fit()

plt.figure()
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='r', zorder=10)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], ecolor='silver', label='Raw')
plt.plot(iv_data['V'], fit_data_trim.fit_function(iv_data['V']), color='orange', label=r'$T_e$ = {:.3g} eV'.format(fit_data_trim.get_temp()), zorder=10)
plt.legend()
plt.ylim(-0.2, 1.3)
plt.show()

In [20]:
# Comparison of different trims
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)

plt.sca(ax0)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], color='silver', ecolor='silver', label='Raw')
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='green', zorder=10)
plt.axvline(x=np.max(fit_data.raw_x), linewidth=0.75, color='green', linestyle='--')

colours = ['red', 'blue', 'orange']

for i, trim_end in enumerate([0.4, 0.6, 0.8]):
    iv_data_trim = ivd.IVData.non_contiguous_trim(iv_data, np.where(iv_data['I'] >= trim_end))
    fit_data_trim = iv_data_trim.multi_fit()
    plt.axvline(x=np.max(iv_data_trim['V']), linewidth=0.75, color=colours[i], linestyle='--')
    plt.plot(iv_data_trim['V'], fit_data_trim.fit_function(iv_data_trim['V']), color=colours[i], label=r'$T_e$ = {:.3g} eV'.format(fit_data_trim.get_temp()), zorder=10)
plt.legend()
# plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-60, 0)

plt.sca(ax1)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], color='silver', ecolor='silver', label='Raw')
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='green', zorder=10)
plt.axvline(x=np.max(fit_data.raw_x), linewidth=0.75, color='green', linestyle='--')
plt.axvline(x=np.max(iv_data_trim['V']), linewidth=0.75, color=colours[i], linestyle='--')
plt.plot(iv_data['V'], fit_data_trim.fit_function(iv_data['V']), color=colours[i], label=r'$T_e$ = {:.3g} eV'.format(fit_data_trim.get_temp()), zorder=10)
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-60, 0)
plt.legend()
plt.show()

In [28]:
# Implement a version of Jamie Gunn's 4-parameter fit by subtracting the straight line of the ion saturation region from the whole IV curve. 
# This generates 4 fits: averaged IV, straight line overlay, corrected IV, 3-param fit to corrected IV

# define a straight section and trim the iv data to it
str_sec = np.where(iv_data['V'] <= -40)
iv_data_ss = ivd.IVData.non_contiguous_trim(iv_data, str_sec)
str_sec_end = np.argmax(iv_data['V'][str_sec]) # needed to define the area of the straight section on a graph with a vertical line

# fit & plot a straight line to the 'straight section'
sl_fitter = fts.StraightLineFitter()
fit_data_ss = sl_fitter.fit(iv_data_ss['V'], iv_data_ss['I'], sigma=iv_data_ss['sigma'])

# Extrapolate the straight line over a wider voltage range for illustrative purposes
sl_range = np.linspace(-120, 100, 100)
sl_function = fit_data_ss.fit_function(sl_range)

# Subtract the gradient of the straight section from the whole IV curve.
iv_data_corrected = iv_data.copy()
iv_data_corrected['I'] = iv_data_corrected['I'] - (fit_data_ss.get_param('m') * iv_data_corrected['V'])

fit_data_corrected = iv_data_corrected.multi_fit(fitter=simple_iv_fitter)

plt.figure()
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data['sigma'], label='Full IV', color='darkgrey', ecolor='lightgray')
# plt.plot(sl_range, sl_function, label='SE Line', color='green', linewidth=0.5)
# plt.plot(iv_data_ss['V'], iv_data_ss['I'], label='Straight Section')
# plt.axvline(x=iv_data_ss['V'][0], label='Straight Section', color='green', linewidth=0.5)
# plt.axvline(x=iv_data_ss['V'][str_sec_end], color='green', linewidth=0.5)
# plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV', yerr=iv_data_corrected[c.SIGMA], color='gray', ecolor='lightgray')
# plt.plot(*fit_data_corrected.get_fit_plottables(), label='3 Param-Fit', zorder=10, color='r')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data['sigma'], label='Full IV', color='darkgrey', ecolor='lightgray')
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5, zorder=10)
# plt.plot(iv_data_ss['V'], iv_data_ss['I'], label='Straight Section')
# plt.axvline(x=iv_data_ss['V'][0], label='Straight Section', color='green', linewidth=0.5)
# plt.axvline(x=iv_data_ss['V'][str_sec_end], color='green', linewidth=0.5)
# plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV', yerr=iv_data_corrected[c.SIGMA], color='gray', ecolor='lightgray')
# plt.plot(*fit_data_corrected.get_fit_plottables(), label='3 Param-Fit', zorder=10, color='r')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
# plt.plot(iv_data['V'], iv_data['I'], label='Full IV', color='darkgrey')
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5, zorder=10)
# plt.plot(iv_data_ss['V'], iv_data_ss['I'], label='Straight Section')
# plt.axvline(x=iv_data_ss['V'][0], label='Straight Section', color='green', linewidth=0.5)
# plt.axvline(x=iv_data_ss['V'][str_sec_end], color='green', linewidth=0.5)
plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV', yerr=iv_data_corrected[c.SIGMA], color='darkgrey', ecolor='lightgray')
# plt.plot(*fit_data_corrected.get_fit_plottables(), label='3 Param-Fit', zorder=10, color='r')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
# plt.plot(iv_data['V'], iv_data['I'], label='Full IV', color='darkgrey')
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5)
# plt.plot(iv_data_ss['V'], iv_data_ss['I'], label='Straight Section')
# plt.axvline(x=iv_data_ss['V'][0], label='Straight Section', color='green', linewidth=0.5)
# plt.axvline(x=iv_data_ss['V'][str_sec_end], color='green', linewidth=0.5)
plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV', yerr=iv_data_corrected[c.SIGMA], color='darkgrey', ecolor='lightgray')
plt.plot(*fit_data_corrected.get_fit_plottables(), label='3 Param-Fit', zorder=10, color='r')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

(-102, 5)

In [12]:
# Create new averaged iv figure

A_coll_0 = probe.get_collection_area(np.radians(9.95))
d_A_coll = np.abs(probe.get_collection_area(np.radians(9.95) + np.radians(0.8)) - A_coll_0)

v_f_fitted = fit_data.get_param('V_f')
# T_e_fitted = fit_data[c.ELEC_TEMP]
# a_fitted = fit_data[c.SHEATH_EXP]
# I_sat_fitted = fit_data[c.ION_SAT]

d_v_f_fitted = fit_data.get_param('V_f', errors_fl=True).error
# d_T_e_fitted = iv_data[c.ERROR_STRING.format(c.ELEC_TEMP)].values[0]
# d_a_fitted = iv_data[c.ERROR_STRING.format(c.SHEATH_EXP)].values[0]
# d_I_sat_fitted = iv_data[c.ERROR_STRING.format(c.ION_SAT)].values[0]

v_f_approx = - 3 * fit_data.get_temp()
d_v_f_approx = 0.05 * v_f_approx

c_s_fitted = lp.sound_speed(fit_data.get_temp(), gamma_i=1)
d_c_s_fitted = lp.d_sound_speed(c_s_fitted, fit_data.get_temp(), fit_data.get_temp(errors_fl=True).error)
n_e_fitted = lp.electron_density(fit_data.get_isat(), c_s_fitted, A_coll_0)
d_n_e_fitted = lp.d_electron_density(n_e_fitted, c_s_fitted, d_c_s_fitted, A_coll_0, d_A_coll, fit_data.get_isat(),
                                     fit_data.get_isat(errors_fl=True).error)

In [23]:
print(f"iv = averaged: \n"
      f"\t v_f = {v_f_fitted:.3g} +- {d_v_f_fitted:.1g} \n"
      f"\t T_e = {fit_data.get_temp():.3g} +- {fit_data.get_temp(errors_fl=True).error:.1g} \n"
      f"\t I_sat = {fit_data.get_isat():.3g} +- {fit_data.get_isat(errors_fl=True).error:.1g} \n"
      f"\t n_e = {n_e_fitted:.3g} +- {d_n_e_fitted:.1g} \n"
      f"\t a = {fit_data.get_sheath_exp():.3g} +- {fit_data.get_sheath_exp(errors_fl=True).error:.1g} \n"
      f"\t c_s = {c_s_fitted:.3g} +- {d_c_s_fitted:.1g} \n"
      f"\t A_coll = {A_coll_0:.3g} +- {d_A_coll:.1g} \n")

iv = averaged: 
	 v_f = -13.5 +- 0.01 
	 T_e = 6.08 +- 0.02 
	 I_sat = 1.08 +- 0.001 
	 n_e = 7.21e+19 +- 5e+18 
	 a = 0.00656 +- 0.0003 
	 c_s = 3.41e+04 +- 7e+01 
	 A_coll = 2.75e-06 +- 2e-07 



In [55]:
I_f = probe.get_analytical_iv(fit_data.raw_x, v_f_fitted, np.radians(9.95), fit_data.get_temp(), n_e_fitted,
                                print_fl=True)
I_ts = probe.get_analytical_iv(fit_data.raw_x, v_f_approx, np.radians(9.95), T_e_ts, n_e_ts,
                                 print_fl=True)

plt.figure()
plt.errorbar(fit_data.raw_x, fit_data.raw_y, yerr=fit_data.sigma,
             label='Raw IV', ecolor='silver', color='gray', zorder=-1)
# plt.plot(iv_data[c.RAW_X].tolist()[0], I_f, label='Analytical - measured', linestyle='dashed', linewidth=1, color='r')
plt.plot(fit_data.raw_x, fit_data.fit_y, color='blue', linewidth=1.2,
         label='Fit - ({:.2g}eV, {:.2g}m'.format(fit_data.get_temp(), n_e_fitted)+r'$^{-3}$)')
plt.plot(fit_data.raw_x, I_ts, linestyle='dashed', color='red',
         label='Analytical from TS - ({:.2g}eV, {:.2g}m'.format(T_e_ts, n_e_ts)+'$^{-3}$)')

plt.legend()
# plt.title('Comparison of analytical to measured IV curves for the small area probe')
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim([-0.35, 1.55])

a = 0.0044868996572897165, c_s = 34125.60926134689, lambda_d = 2.1585802219076544e-06, I_0 = 1.0835414889505426
a = 0.00165323075130103, c_s = 17582.941645962666, lambda_d = 7.953445529387089e-07, I_0 = 1.0917034253527582


(-0.35, 1.55)

In [29]:
# Two temp illustrative fit
# Implement a two temp fitter with different saturation currents (basically two IVs stacked on top of each other)
two_temp_fitter = fts.TwoTempImpurityIVFitter()
two_temp_fitter.set_fixed_values({c.ELEC_TEMP: 1.6, 'T2': 6})

# iv_trim = ivd.IVData.fractional_trim(iv_data_corrected, trim_beg=0.05, trim_end=0.85)
float_trim_indices = -iv_data_corrected['I'] < 0
iv_float_trim = ivd.IVData.non_contiguous_trim(iv_data_corrected, float_trim_indices)

# Set initial fit guess parameters
init_vals = [0.85, 1.5, -15, 0.2, 8, -15]
approx_curve = two_temp_fitter.fit_function(iv_float_trim['V'], *init_vals)
iv_0_vals = init_vals[:3]
iv_1_vals = init_vals[3:]

fit_float_trim = two_temp_fitter.fit_iv_data(iv_float_trim, initial_vals=init_vals)
fit_0_vals = fit_float_trim.fit_params.get_values()[:3]
fit_1_vals = fit_float_trim.fit_params.get_values()[3:]

# Create initial plot
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)

plt.sca(ax0)
plt.axhline(y=0, color='gray', linewidth=0.5)
# plt.axvline(x=0, color='gray', linewidth=0.5)
plt.errorbar(iv_float_trim['V'], iv_float_trim['I'], yerr=iv_float_trim['sigma'], ecolor='silver', color='grey', label='Averaged')
plt.errorbar(*fit_float_trim.get_fit_plottables(), label='Fitted', color='red')
plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *fit_0_vals), label='Fit-IV-0', linewidth=0.75, color='orange')
plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *fit_1_vals), label='Fit-IV-1', linewidth=0.75, color='orangered')
plt.legend()
# plt.xlabel(r'$V_p$ / V')
plt.ylabel('I / A')
plt.show()
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.sca(ax1)
plt.errorbar(iv_float_trim['V'], iv_float_trim['I'], yerr=iv_float_trim['sigma'], ecolor='silver', color='grey', label='Averaged')
plt.plot(iv_float_trim['V'], approx_curve, label='Approx fit', color='blue')
plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *iv_0_vals), label='IV-0', linewidth=0.75, color='darkblue')
plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *iv_1_vals), label='IV-1', linewidth=0.75, color='steelblue')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel('I / A')
plt.show()
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

(-102, 5)

In [14]:
from matplotlib.widgets import Slider, Button, RadioButtons

# Implement a two temp fitter with different saturation currents (basically two IVs stacked on top of each other)

# Set initial fit guess parameters
init_vals = [0.85, 1.5, -15, 0.2, 8, -15]
approx_curve = fts.two_temp_fitter.fit_function(iv_float_trim['V'], *init_vals)
iv_0_vals = init_vals[:3]
iv_1_vals = init_vals[3:]

# Create initial plot with space at the bottom for sliders
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.3)

plt.axhline(y=0, color='gray', linewidth=0.5)
plt.errorbar(iv_float_trim['V'], iv_float_trim['I'], yerr=iv_float_trim['sigma'], ecolor='silver', color='grey', label='Averaged', zorder=-1)
handle1, = plt.plot(iv_float_trim['V'], approx_curve, label='Approx fit', color='blue')
handle2, = plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *iv_0_vals), label='IV-0', linewidth=0.75, color='darkblue')
handle3, = plt.plot(iv_data['V'], simple_iv_fitter.fit_function(iv_data['V'], *iv_1_vals), label='IV-1', linewidth=0.75, color='steelblue')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel('I / A')
plt.show()
plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

# Create and populate sliders
axcolor = 'lightgoldenrodyellow'
axvf1 = plt.axes([0.2, 0.03, 0.55, 0.03], facecolor=axcolor)
axt1 = plt.axes([0.2, 0.06, 0.55, 0.03], facecolor=axcolor)
axisat1 = plt.axes([0.2, 0.09, 0.55, 0.03], facecolor=axcolor)

axvf2 = plt.axes([0.2, 0.14, 0.55, 0.03], facecolor=axcolor)
axt2 = plt.axes([0.2, 0.17, 0.55, 0.03], facecolor=axcolor)
axisat2 = plt.axes([0.2, 0.20, 0.55, 0.03], facecolor=axcolor)

svf1 = Slider(axvf1, r'$V_{f1}$', -30.0, 10.0, valinit=init_vals[2], valstep=0.25)
svf2 = Slider(axvf2, r'$V_{f2}$', -30.0, 10.0, valinit=init_vals[5], valstep=0.25)
st1 = Slider(axt1, r'$T_{e1}$', 0, 15, valinit=init_vals[1], valstep=0.1)
st2 = Slider(axt2, r'$T_{e2}$', 0, 15, valinit=init_vals[4], valstep=0.1)
si1 = Slider(axisat1, r'$I_{sat,1}$', 0, 1.1, valinit=init_vals[0], valstep=0.05)
si2 = Slider(axisat2, r'$I_{sat,2}$', 0, 1.1, valinit=init_vals[3], valstep=0.05)

def update(val):
    v_f1 = svf1.val
    v_f2 = svf2.val
    t_1 = st1.val
    t_2 = st2.val
    isat_1 = si1.val
    isat_2 = si2.val
    
    new_init_vals = [isat_1, t_1, v_f1, isat_2, t_2, v_f2]
    iv1_vals = [isat_1, t_1, v_f1]
    iv2_vals = [isat_2, t_2, v_f2]
    handle1.set_ydata(two_temp_fitter.fit_function(iv_float_trim['V'], *new_init_vals))
    handle2.set_ydata(simple_iv_fitter.fit_function(iv_data['V'], *iv1_vals))
    handle3.set_ydata(simple_iv_fitter.fit_function(iv_data['V'], *iv2_vals))
    fig.canvas.draw_idle()
    
svf1.on_changed(update)
svf2.on_changed(update)
st1.on_changed(update)
st2.on_changed(update)
si1.on_changed(update)
si2.on_changed(update)

fitax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(fitax, 'Fit', color=axcolor, hovercolor='0.975')

def plot_fit(event):
    v_f1 = svf1.val
    v_f2 = svf2.val
    t_1 = st1.val
    t_2 = st2.val
    isat_1 = si1.val
    isat_2 = si2.val
    
    new_init_vals = [isat_1, t_1, v_f1, isat_2, t_2, v_f2]
    fit_data_tti = two_temp_fitter.fit_iv_data(iv_float_trim, initial_vals=new_init_vals)
    handle1.set_ydata(fit_data_tti.fit_y)
    
    fit_vals = fit_data_tti.fit_params.get_values()
    si1.set_val(fit_vals[0])
    st1.set_val(fit_vals[1])
    svf1.set_val(fit_vals[2])
    si2.set_val(fit_vals[3])
    st2.set_val(fit_vals[4])
    svf2.set_val(fit_vals[5])
    update(event)
    
button.on_clicked(plot_fit)

plt.show()

AttributeError: module 'flopter.core.fitters' has no attribute 'two_temp_fitter'

In [37]:
# Do some logging

current_offset = fit_data.get_isat() + 0.1
# current_offset = 0

# Use iv_float_trim as it is the corrected IV from the Gunn fit but trimmed down to the floating potential.
log_plot_roi_1 = np.where((iv_float_trim['V'] >= -45) & (iv_float_trim['V'] <= -30))
log_plot_roi_2 = np.where((iv_float_trim['V'] >= -25) & (iv_float_trim['V'] <= -18))
# print(len(log_plot_roi), log_plot_roi)

fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2, 4, figsize=(10,6))
ax1.errorbar(iv_float_trim['V'], -iv_float_trim['I'] + current_offset, yerr=iv_float_trim['sigma'], ecolor='silver')
ax1.plot(iv_float_trim['V'][log_plot_roi_1], -iv_float_trim['I'][log_plot_roi_1] + current_offset)
ax1.plot(iv_float_trim['V'][log_plot_roi_2], -iv_float_trim['I'][log_plot_roi_2] + current_offset)

ax2.plot(iv_float_trim['V'], np.log(-iv_float_trim['I'] + current_offset))
ax2.plot(iv_float_trim['V'][log_plot_roi_1], np.log(-iv_float_trim['I'][log_plot_roi_1] + current_offset))
ax2.plot(iv_float_trim['V'][log_plot_roi_2], np.log(-iv_float_trim['I'][log_plot_roi_2] + current_offset), color='yellow')

ax3.plot(iv_float_trim['V'][log_plot_roi_1], np.log(-iv_float_trim['I'][log_plot_roi_1] + current_offset))

ax4.plot(iv_float_trim['V'][log_plot_roi_2], np.log(-iv_float_trim['I'][log_plot_roi_2] + current_offset))

# CONTESTANTS, START YOUR FITTING

sl_fitter = fts.StraightLineFitter()

# Fit first straight region and then subtract from the log
fit_data_log_1 = sl_fitter.fit(iv_float_trim['V'][log_plot_roi_2], np.log(-iv_float_trim['I'][log_plot_roi_2] + current_offset))
iv_log_corrected = iv_float_trim.copy()
iv_log_corrected['I'] = iv_log_corrected['I'] - (fit_data_log_1.get_param('m') * iv_log_corrected['V'])
iv_log_corrected['I'] = iv_log_corrected['I'] - fit_data_log_1.fit_function(iv_log_corrected['V'])

ax5.plot(iv_float_trim['V'], np.log(-iv_float_trim['I'] + current_offset), color='lightgrey')
ax5.plot(np.linspace(-105,40,100), fit_data_log_1.fit_function(np.linspace(-105,40,100)), 
         label=r'$T_e$ = {:.2g}'.format(1/fit_data_log_1.get_param('m') / 4), color='r', linewidth=0.5, zorder=10)
ax5.plot(np.linspace(-105,40,100), sl_fitter.fit_function(np.linspace(-105,40,100), 1/1.45, 15), 
         label=r'$T_e$ = {:.2g}'.format(1.45), color='b', linewidth=0.5, zorder=10)
ax5.plot(iv_log_corrected['V'], np.log(-iv_log_corrected['I'] + current_offset))
ax5.plot(iv_log_corrected['V'][log_plot_roi_2], np.log(-iv_log_corrected['I'][log_plot_roi_2] + current_offset))
ax5.set_xlim(-50, -10)
ax5.set_ylim(-2.3, 0.85)
ax5.legend()

# log_plot_roi_2 = np.where((iv_log_corrected['V'] >= -25) & (iv_log_corrected['V'] <= -22) & (np.isfinite(np.log(-iv_log_corrected['I'] + current_offset))))

# fit_data_log_2 = sl_fitter.fit(iv_log_corrected['V'][log_plot_roi_2], np.log(-iv_log_corrected['I'][log_plot_roi_2] + current_offset))

# ax6.plot(iv_log_corrected['V'], np.log(-iv_log_corrected['I'] + current_offset))
# ax6.plot(*fit_data_log_2.get_raw_plottables())
# ax6.plot(*fit_data_log_2.get_fit_plottables())

plt.tight_layout()
# fig = plt.figure()
# fit_data_log.plot(fig=fig, show_fl=False)
# plt.plot(trim_sweep_avg_up['voltage'].data, sig.savgol_filter(np.log(-trim_sweep_avg_up['current'] + current_offset), 21, 4),
#          label='Smoothed log')
# plt.legend()


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in log
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in log


## Attempt to use PlasmaPy to get the EEDF from the averaged IV characteristic

In [41]:
from plasmapy.diagnostics.langmuir import *
from plasmapy.atomic import Particle
import astropy.units as u

In [123]:
print(iv_float_trim['V'].dtype, iv_float_trim['I'])

iv_characteristic = Characteristic(iv_data['V'][::2] * u.V, -iv_data['I'][::2] * u.A)
iv_characteristic.check_validity()
print(A_coll_0)

iv_characteristic.plot()

float64 [1.0540719  1.05400223 1.05850644 ... 1.04552538 1.05108518 1.05339837]
2.748745727715695e-06


In [100]:
# Load the bias and current values stored in the .p pickle file.
bias, current = np.load('/home/jleland/Coding/Jupyter/Beckers2017.npy')
print(bias.dtype)

# Create the Characteristic object, taking into account the correct units
characteristic = Characteristic(( 2* np.array(bias) - 10) * u.V,
                                (80 * np.array(current)*1e3 - 1000) * u.mA)

# Calculate the cylindrical probe surface area
ex_probe_length = 1.145 * u.mm
ex_probe_diameter = 1.57 * u.mm
ex_probe_area = (probe_length * np.pi * probe_diameter +
              np.pi * 0.25 * probe_diameter**2)

# characteristic.plot()

float64


In [130]:
characteristic.plot()
trim_ch = characteristic[-10:]
plt.plot(trim_ch.bias, trim_ch.current)

In [116]:
def noisify(input_arr, noise_scale=0.01):
    return (noise_scale * (np.random.rand(len(input_arr.value)) - 0.5)) + input_arr.value

In [150]:
print(trim_ch.bias[0] + iv_characteristic.bias[-1])
new_bias = np.concatenate([iv_characteristic.bias, noisify(trim_ch.bias - trim_ch.bias[0] + iv_characteristic.bias[-1])])
new_current = np.concatenate([iv_characteristic.current, noisify(trim_ch.current - trim_ch.current[0] + iv_characteristic.current[-1])])

plt.figure()
plt.plot(new_bias, new_current)

iv_characteristic = Characteristic(new_bias.value * u.V, new_current.value * u.A)

6.095766795435047 V


In [147]:
from scipy.signal import savgol_filter

print(len(iv_data['V']))

sm_bias = savgol_filter(iv_data['V'][:2500:2], 61, 10)
sm_current = savgol_filter(iv_data['I'][:2500:2], 61, 10)

plt.figure()
plt.plot(iv_data['V'], -iv_data['I'], label='Raw')
plt.plot(sm_bias, -sm_current, label='Smoothed')
plt.legend()

iv_characteristic = Characteristic(sm_bias * u.V, -sm_current * u.A)

5000


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [149]:
print(swept_probe_analysis(iv_characteristic, A_coll_0 * u.m**2, 1 * u.u, visualize=True, plot_EEDF=True))

{'V_P': <Quantity 5.71195522 V>, 'V_F': <Quantity -12.89779388 V>, 'I_es': <Quantity 1.96073392 A>, 'I_is': <Quantity -1.16143525 A>, 'n_e': <Quantity 1.09618621e+20 1 / m3>, 'n_i': <Quantity 1.84331493e+21 1 / m3>, 'T_e': <Quantity 0.0589301 eV>, 'n_i_OML': <Quantity 4.86597269e+19 1 / m3>}
